In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)

In [ ]:
name_projects = []
name_projects.append('project/output_FJSJ_17-02/')
name_projects.append('project/output_FJSJ_17-03/')
name_projects.append('project/output_FJSJ_17-04/')
name_projects.append('project_repartition_v3.0/output_repar_01-01/')
name_projects.append('project_repartition_v3.0/output_repar_01-02/')

In [ ]:
name_project_sample = 'project_repartition_v3.0/output_repar_01-01/'
dir_project_sample = dir_project_workspace + name_project_sample
filename = dir_project_sample+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
locinfo_sample = {}
locinfo_sample[name_project_sample] = pd.read_excel(dir_project_sample + 'subwork_location.xlsx',sheet_name='location')
key_subworks = info_basic['key_subworks']
key_subworks

In [ ]:
key_subworks = ['48-13--464']

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [ ]:
stainfo_alls = {}
for name_project in name_projects:
    dir_project = dir_project_workspace + name_project
    filename = dir_project+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    #dir_ds = dir_project + info_basic['dir_ds']
    stalistname = dir_project+info_basic['stalistname']
    stainfo_alls[name_project] = pd.read_excel(stalistname,sheet_name=None)

In [ ]:
locinfo_alls = {}
for name_project in name_projects:
    dir_project = dir_project_workspace + name_project
    locinfo_alls[name_project] = pd.read_excel(dir_project + 'subwork_location.xlsx',sheet_name='location')

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
dir_image = dir_project_workspace + 'compare_highermode/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)

In [ ]:
locinfo_sample[name_project_sample]['key_subwork'].tolist()

In [ ]:
def find_nearset(key_subwork,name_project):
	global locinfo_alls
	global name_project_sample
	global locinfo_sample
	global key_subworks
	index = locinfo_sample[name_project_sample]['key_subwork'].tolist().index(key_subwork)
	lon_this = locinfo_sample[name_project_sample]['lon_centroid'][index]
	lat_this = locinfo_sample[name_project_sample]['lat_centroid'][index]
	lon_all = []
	lat_all = []
	
	lon_all = locinfo_alls[name_project]['lon_centroid'].tolist()
	lat_all = locinfo_alls[name_project]['lat_centroid'].tolist()
	key_all = locinfo_alls[name_project]['key_subwork'].tolist()

	dist = np.sqrt((np.array(lon_all)-lon_this)**2+(np.array(lat_all)-lat_this)**2)
	indx = np.argsort(dist)
	key_nearset = key_all[indx[0]]

	return str(key_nearset)

In [ ]:
c_map = 'jet'
v_min = 0.1
v_max = None
xlim_f = [1,25]
for key_subwork in key_subworks:
    print('start ',str(key_subwork),' ',str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))
    fig,ax = plt.subplots(2,len(name_projects),figsize=(5*len(name_projects),9))
    for name_project in name_projects:
        key_this_find = find_nearset(key_subwork,name_project)
        stainfo_all = stainfo_alls[name_project]
        col_index = name_projects.index(name_project)
        dir_project = dir_project_workspace + name_project
        filename = dir_project+'Basic_info.yml'
        with open(filename, 'r', encoding='utf-8') as f:
            info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
        dir_ds = dir_project + info_basic['dir_ds']
        stainfo = stainfo_all[str(key_this_find)]
        print('key_this_find: ',key_this_find)
        stalist = stainfo['Station'].tolist()
        lat = stainfo['latitude'].tolist() 
        lon = stainfo['longitude'].tolist()
        ax[1][col_index]=plotlib.plot_area(ax[1][col_index],lon_all,lat_all,lon,lat)
        for i in range(len(faults)):
            ax[1][col_index].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
        
        title0 = name_project.split('/')[-2] + ' ' +str(key_this_find)
        c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
        ds = h5py.File(dir_ds+'ds_'+str(key_this_find)+'.h5', 'r')
        ds_remove = ds['ds_remove'][0]
        ds_linear = ds['ds_linear'][0]
        f = ds['f'][:]
        ax[0][col_index] = plotlib.plot_fj(ax[0][col_index] ,ds_remove,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
        ax[0][col_index] .set_xlim(xlim_f)
        #ax[1][col_index] = plotlib.plot_fj(ax[1][col_index] ,ds_linear,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
        #ax[1][col_index] .set_xlim(xlim_f)
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'.png',dpi=60)
    #plt.close()